In [1]:
from dotenv import load_dotenv
import os
import dspy
load_dotenv()

/Users/viktorsvan/Python_koder/ingenjor_A_och_maskininlarnings_kod/LLM-DSPy-project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'), temperature=1)
dspy.configure(lm=lm)


lm("What year is it?") 

['The current year is 2023.']

Signatures

Strukturera vad LMM kommer göra:

In [3]:
class Chat(dspy.Signature):
    "You are a helpful assistant."
    
    question: str = dspy.InputField(desc="Questions asked by the user")
    response: str = dspy.OutputField(desc="Response to the question")

Modules:

Här programmerar vi olika prompting tekniker till våra signatures istället för att skriva dom:

In [4]:
class Model(dspy.Module):
    def __init__(self):
        super().__init__()
        self.respond = dspy.Predict(Chat)

    def forward(self, question: str):
        return self.respond(question=question)

In [5]:
model = Model()
response = model(question="Why is the earth round?")

In [6]:
lm.inspect_history()





[2025-05-19T16:30:21.472415]

System message:

Your input fields are:
1. `question` (str): Questions asked by the user
Your output fields are:
1. `response` (str): Response to the question
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are a helpful assistant.


User message:

[[ ## question ## ]]
Why is the earth round?

Respond with the corresponding output fields, starting with the field `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## response ## ]]
The Earth is round primarily due to the force of gravity. As the planet formed, gravity pulled material towards its center, causing it to take on a shape that allows for the most efficient distribution of mass: a sphere. This shape minimizes potential energy and allows for a balan

In [7]:
class Model(dspy.Module):
    def __init__(self):
        super().__init__()
        self.respond = dspy.ChainOfThought(Chat)

    def forward(self, question: str):
        return self.respond(question=question)

In [8]:
class Model(dspy.Module):
    def __init__(self):
        super().__init__()
        self.respond = dspy.ChainOfThought(Chat)

    def forward(self, question: str):
        return self.respond(question=question)

In [9]:
model = Model()
response = model(question="Why is the earth round?")

In [10]:
lm.inspect_history()





[2025-05-19T16:30:21.531147]

System message:

Your input fields are:
1. `question` (str): Questions asked by the user
Your output fields are:
1. `reasoning` (str)
2. `response` (str): Response to the question
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are a helpful assistant.


User message:

[[ ## question ## ]]
Why is the earth round?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## reasoning ## ]]
The Earth is round due to the gravitational forces acting upon it. When a planet forms, its mass draws matter toward its center, leading to a shape that minimizes potential energy. This shape is

Evaluators:

För att mäta hur bra din modell presterar till din task behöver du specificera metrics, men först ha data som den kan evalueras mot - DSPy exapmles

Dataset från Kaggle om terapi. 

Beskrivning: "This dataset is a curated collection of questions and answers sourced from two prominent online counseling and therapy platforms."

In [11]:
import kagglehub

path = kagglehub.dataset_download("melissamonfared/mental-health-counseling-conversations-k")

path += "/combined_dataset.json"

In [12]:
import pandas as pd

df = pd.read_json(path, lines=True)
df = df.sample(frac=1, random_state=30).reset_index(drop=True)

print(df.shape)

(3512, 2)


In [13]:
df = df[:200]

print(df.head(3))
print(df.tail(3))

                                             Context  \
0  When my daughter is stressed about a silly thi...   
1  I've been going through a rough time lately. I...   
2  I'm in my early 20s, and I've been seeing my b...   

                                            Response  
0  I agree with your observation about your daugh...  
1  Hi Brookfield, It can be unsettling when we fe...  
2  Hello, and thank you for your question. I am v...  
                                               Context  \
197  I'm always listening to my husband, but it fee...   
198  I'm a female in my mid 20s. Lately I tend to o...   
199  A lot of times, I avoid situations where I am ...   

                                              Response  
197  Have the two of you ever discussed how you fee...  
198  Speaking with a licensed therapist will help y...  
199  Why not accept and tolerate that you naturally...  


In [14]:
df.columns = ['question', 'response']
df

question  \
0    When my daughter is stressed about a silly thi...   
1    I've been going through a rough time lately. I...   
2    I'm in my early 20s, and I've been seeing my b...   
3                After I told them, they yelled at me.   
4    I start counseling/therapy in a few days (I'm ...   
..                                                 ...   
195  I start counseling/therapy in a few days (I'm ...   
196  My girlfriend and I have broken up and gotten ...   
197  I'm always listening to my husband, but it fee...   
198  I'm a female in my mid 20s. Lately I tend to o...   
199  A lot of times, I avoid situations where I am ...   

                                              response  
0    I agree with your observation about your daugh...  
1    Hi Brookfield, It can be unsettling when we fe...  
2    Hello, and thank you for your question. I am v...  
3    It sounds like your family responded out of fe...  
4    Please feel free to cry during therapy if you ...  
..                                                 ...  
195  Please feel free to cry during therapy if you ...  
196  Love is not enough to keep a relationship toge...  
197  Have the two of you ever discussed how you fee...  
198  Speaking with a licensed therapist will help y...  
199  Why not accept and tolerate that you naturally...  

[200 rows x 2 columns]

In [15]:
data = df.to_dict(orient='records')
data = [dspy.Example(**d).with_inputs('question') for d in data]

In [16]:
example = data[0]
example

Example({'question': "When my daughter is stressed about a silly thing from school, she starts crying and freaking out. She is a bright student, always has a 4.0, but I am afraid she is stressing too much. I’m afraid it’s going to break her. I don't know if I should get her to a doctor or someone because this is not normal.", 'response': "I agree with your observation about your daughter feeling stressed. \xa0Are you able to open this topic in conversation with her?Also, reflect on your own expectations as a parent. \xa0It is possible that your daughter is trying to please you by getting consistently high grades.If your daughter prefers talking in confidence to a therapist, then this may help her regain a sense of balance in her life so that schoolwork feels less stressful.I wouldn't take her to a doctor because based on what you write, the problem is psychological and emotionally based. \xa0While the stress may have physical symptoms, addressing the root cause of the problem has nothi

Till skillnad från vanlig ML träning: Så ska man dela upp datasetet med 20 procent till träning och 80 procent till validering. 


In [17]:
trainset = data[:40]   # 20% 
valset = data[40:200]  # 80% 
    
len(trainset), len(valset)

(40, 160)

SemanticF1 är en evaluation metric som DSPy erbjuder och är som en "LLM judge" som kollar på hur mycket nyckelfaktorerna i det riktiga svaret stämmer överens med predictionen och hur mycket tar den upp som inte är med i riktiga svaret. Threshold avgör hur likt det måste vara.

Underliggande kod:
https://github.com/stanfordnlp/dspy/blob/main/dspy/evaluate/auto_evaluation.py#L21


In [18]:
from dspy.evaluate import SemanticF1
metric = SemanticF1(threshold=0.66)

In [19]:
pred = model(example.inputs())


score = metric(example, pred)

print(f"Question: \t {example.question}\n")
print(f"Gold Response: \t {example.response}\n")
print(f"Predicted Response: \t {pred.response}\n")
print(f"Semantic F1 Score: {score:.2f}")

Question: 	 When my daughter is stressed about a silly thing from school, she starts crying and freaking out. She is a bright student, always has a 4.0, but I am afraid she is stressing too much. I’m afraid it’s going to break her. I don't know if I should get her to a doctor or someone because this is not normal.

Gold Response: 	 I agree with your observation about your daughter feeling stressed.  Are you able to open this topic in conversation with her?Also, reflect on your own expectations as a parent.  It is possible that your daughter is trying to please you by getting consistently high grades.If your daughter prefers talking in confidence to a therapist, then this may help her regain a sense of balance in her life so that schoolwork feels less stressful.I wouldn't take her to a doctor because based on what you write, the problem is psychological and emotionally based.  While the stress may have physical symptoms, addressing the root cause of the problem has nothing to do directl

In [20]:
from dspy.evaluate import Evaluate

evaluator = Evaluate(devset=trainset, num_threads=1, display_progress=True, display_table=5)

evaluator(model, metric=metric)

Average Metric: 21.47 / 40 (53.7%): 100%|██████████| 40/40 [00:00<00:00, 631.64it/s]

2025/05/19 16:30:22 INFO dspy.evaluate.evaluate: Average Metric: 21.46711643824144 / 40 (53.7%)


question  \
0  When my daughter is stressed about a silly thing from school, she ...   
1  I've been going through a rough time lately. I been into nothing b...   
2  I'm in my early 20s, and I've been seeing my boyfriend for a year ...   
3                                  After I told them, they yelled at me.   
4  I start counseling/therapy in a few days (I'm freaking out) but my...   

                                                        example_response  \
0  I agree with your observation about your daughter feeling stressed...   
1  Hi Brookfield, It can be unsettling when we feel something as fund...   
2  Hello, and thank you for your question. I am very sorry that you a...   
3  It sounds like your family responded out of fear! They may need so...   
4  Please feel free to cry during therapy if you suddenly feel painfu...   

                                                               reasoning  \
0  It's understandable to be concerned when a child exhibits signs of...   
1  It's understandable to feel upset and confused about discovering n...   
2  It's important to recognize that comments about appearance, partic...   
3  It seems that the user has experienced a negative reaction from ot...   
4  It's completely normal to feel anxious about starting counseling o...   

                                                           pred_response  \
0  It’s important to acknowledge that your daughter's feelings are va...   
1  It's completely normal to feel confused and scared during this tim...   
2  It sounds like you're feeling hurt by your boyfriend's recent comm...   
3  I'm sorry to hear that you had such a difficult experience. It can...   
4  It's understandable to feel freaked out about starting therapy, bu...   

   SemanticF1  
0  ✔️ [0.769]  
1  ✔️ [0.514]  
2  ✔️ [0.444]  
3              
4  ✔️ [0.740]

53.67

Så default prompten med Chain of Thought löste thresholden med 42% att likna "Golden answers" tillräkligt mycket

Prompt omptimization:

DSPy har flera optimizers man kan använda mellan. En av deras senaste är MIPROv2 som kan användas till både zero-shot och few-shot optimization.

In [21]:
from dspy.teleprompt import MIPROv2

optimizer = MIPROv2(
    metric=metric,
    prompt_model= dspy.LM('openai/gpt-4', api_key=os.getenv('OPENAI_API_KEY')),
    task_model= dspy.LM('openai/gpt-4', api_key=os.getenv('OPENAI_API_KEY')),
    auto="light",
    num_threads=1
)

För att skapa prompt candidatsen(Instruktionerna) används träningsdatan och för att utvärdera dom används validationdatan:

In [22]:
training_loop = optimizer.compile(
    student = model.deepcopy(),
    trainset=trainset,
    valset=valset,
    seed = 9,
    requires_permission_to_run=False
)

2025/05/19 16:30:22 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: True
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 100

2025/05/19 16:30:22 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/05/19 16:30:22 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/05/19 16:30:22 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


 15%|█▌        | 6/40 [00:46<04:21,  7.68s/it]


Bootstrapped 4 full traces after 6 examples for up to 1 rounds, amounting to 6 attempts.
Bootstrapping set 4/6


 12%|█▎        | 5/40 [00:37<04:24,  7.56s/it]


...

2025/05/19 17:30:20 INFO dspy.evaluate.evaluate: Average Metric: 21.930495601983374 / 35 (62.7%)
2025/05/19 17:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 62.66 on minibatch of size 35 with parameters ['Predictor 0: Instruction 1', 'Predictor 0: Few-Shot Set 4'].
2025/05/19 17:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [59.7, 54.26, 64.11, 57.93, 62.71, 57.83, 63.95, 65.21, 62.66]
2025/05/19 17:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [52.68, 65.0]
2025/05/19 17:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 65.0
2025/05/19 17:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/05/19 17:30:20 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 12 / 13 - Minibatch ==



Average Metric: 19.90 / 35 (56.8%): 100%|██████████| 35/35 [11:23<00:00, 19.53s/it]

2025/05/19 17:41:44 INFO dspy.evaluate.evaluate: Average Metric: 19.895053220127036 / 35 (56.8%)
2025/05/19 17:41:44 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 56.84 on minibatch of size 35 with parameters ['Predictor 0: Instruction 1', 'Predictor 0: Few-Shot Set 1'].
2025/05/19 17:41:44 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [59.7, 54.26, 64.11, 57.93, 62.71, 57.83, 63.95, 65.21, 62.66, 56.84]
2025/05/19 17:41:44 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [52.68, 65.0]
2025/05/19 17:41:44 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 65.0
2025/05/19 17:41:44 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/05/19 17:41:44 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 13 / 13 - Full Evaluation =====
2025/05/19 17:41:44 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 63.95) from minibatch trials...



Average Metric: 62.73 / 100 (62.7%): 100%|██████████| 100/100 [08:51<00:00,  5.31s/it]

2025/05/19 17:50:35 INFO dspy.evaluate.evaluate: Average Metric: 62.73391384494633 / 100 (62.7%)
2025/05/19 17:50:35 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [52.68, 65.0, 62.73]
2025/05/19 17:50:35 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 65.0
2025/05/19 17:50:35 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/05/19 17:50:35 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/05/19 17:50:35 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 65.0!


In [23]:
training_loop(question="Latley I have been feeling down, what should I do?").response

"I'm sorry to hear you're feeling down lately. It's important to acknowledge these feelings and not dismiss them. First, consider reaching out to someone you trust, whether it's a friend, family member, or therapist, to talk about how you're feeling; sometimes, sharing can provide relief and perspective. Engaging in activities that you enjoy or that help you relax, like going for a walk, reading, or practicing mindfulness, can also make a difference. Additionally, ensuring you maintain a routine, eat well, and get enough rest can have a positive impact on your mood. If these feelings persist, please consider talking to a mental health professional who can provide support tailored to your needs. Remember, it's okay to ask for help when you need it."

In [24]:
dspy.inspect_history(n=1)





[2025-05-19T17:50:41.471527]

System message:

Your input fields are:
1. `question` (str): Questions asked by the user
Your output fields are:
1. `reasoning` (str)
2. `response` (str): Response to the question
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are an empathetic assistant, trained in Cognitive Behavioral Tactics. A user will present you with a personal question or concern, often involving emotional or psychological issues. Your task is to understand the context and nuances of the user's question, and then generate a response that is not only accurate but also sensitive to the user's concerns. You should provide a thoughtful, reasoned response that offers guidance and support, while also explaining the reasoning behind your response to pr

Resultat:

Förbättrar resultatet från 53% --> 65%
